In [1]:
#### 1065 Calculation #####
import pandas as pd
import numpy as np
import scipy as sc
import scipy.optimize as opt

data_raw = pd.read_csv("Test_data.csv", header=2,low_memory=False)
data_raw = data_raw.convert_objects(convert_numeric=True)
data_un = pd.DataFrame()

## Prepare constant Values
bs = 0.00821722713286476 # Has to be calculated
RFPF = 0.0133
CH4_RF = 1.1
Tchiller = 7
Pamb = 98.0918852
Pchiller = Pamb + 23.75
xCO2intdry = 0.000375
xCO2dildry = 0.000375
xTHC_THC_FID_init = 1
print("Initialization finished!")

Initialization finished!


In [2]:
# Preparation of Species
species_raw = [col for col in list(data_raw) if col.startswith('E_CO')]
if 'E_COHD2' in species_raw:
    CO = 'E_COHD2'
    CO_flag = 1 # CO-High
else:
    CO = 'E_COLD2'
    CO_flag = 0 # CO-Low

In [3]:
##### First Level of Calculation #####
data_un["E_CO2D2"] = data_raw.E_CO2D2*10000
data_un[CO] = data_raw.get(CO)*10000
data_un["E_NOXD2"] = data_raw.E_NOXD2
data_un["E_THCW2"] = data_raw.E_THCW2
data_un["xCH4wet"] = data_raw.E_CH4W2/1000000
data_un["C_FRAIRWS"] = data_raw.C_FRAIRWS
data_un["BARO Press"] = data_raw.P_INLET*100
data_un["T_INLET"] = data_raw.T_INLET
data_un["M_RELHUM"] = data_raw.M_RELHUM
data_un["mfuel"] = data_raw.M_FRFUEL*1000/3600
print("First Level of Calculation finished!")

First Level of Calculation finished!


In [4]:
##### Second Level of Calculation #####
data_un["xCO2meas"] = data_un.E_CO2D2/1000000
data_un["xCOmeas"] = data_un.get(CO)/1000000
data_un["xNOxmeas"] = data_un.E_NOXD2/1000000
data_un["xTHCmeas"] = data_un.E_THCW2/1000000
data_un["Molar Flow Wet"] = data_un.C_FRAIRWS*1000/3600
data_un["maf_kgh (wet)"] = data_un.C_FRAIRWS
data_un["pH2O @ inlet"] = 10**(10.79574*(1-(273.16/(273.15+data_un.T_INLET)))-5.028*np.log10((273.15+data_un.T_INLET)/273.16)+0.000150475*(1-10**(-8.2969*(((273.15+data_un.T_INLET)/273.16)-1)))+0.00042873*(10**(4.76955*(1-(273.16/(273.15+data_un.T_INLET))))-1)-0.2138602)  ## Reference: CFR 1065.564
data_un["xH2O"] = data_un.M_RELHUM*data_un.get("pH2O @ inlet")/(data_un.get("BARO Press")*100) ## Reference: CFR 1065.564
data_un["xTHC[THC_FID]cor"] = data_un.E_THCW2-xTHC_THC_FID_init ## Reference: CFR 1065.660
print("Second Level of Calculation finished!")

Second Level of Calculation finished!


In [6]:
##### Third Level of Calculation #####
data_un.E_THCW2 = data_un.get("xTHC[THC_FID]cor") ## Reference: CFR 1065.660
data_un["xNO2meas"] = data_un.xNOxmeas*0
data_un["xNOmeas"] = data_un.xNOxmeas*1
data_un["xTHCwet"] = data_un.xTHCmeas
data_un["Intake Air flow"] = data_un.get("Molar Flow Wet")/28.96
data_un["Mmix"] = 28.96559*(1-data_un.xH2O)+18.01528*data_un.xH2O
data_un["xH2Oint"] = data_un.xH2O
data_un["xH2Odil"] = data_un.xH2O

# Constant Channels
c = np.zeros((len(data_un),1))
c.fill(3.5)
data_un["xH2Ogas"] = c
c.fill(13.875389)
data_un["Molar Mass of HC Emissions"] = c
c.fill(28.0101)
data_un["Molar Mass of CO Emissions"] = c
c.fill(46.0055)
data_un["Molar Mass of NOx Emissions (as NO2)"] = c
c.fill(44.0095)
data_un["Molar Mass of CO2 Emissions"] = c
c.fill(13.875389)
data_un["Molar Mass of NMHC Emissions"] = c
c.fill(0.866)
data_un["Wc"] = c
c.fill(1.84)
data_un['alpha'] = c
c.fill(0)
data_un['beta'] = c
data_un['gamma'] = c
data_un['delta'] = c
c = None
print("Third Level of Calculation finished!")

Third Level of Calculation finished!


In [7]:
##### Fourth Level of Calculation #####
data_un["xNMHCwet"] = (data_un.xTHCwet-data_un.xCH4wet*CH4_RF)/(1-RFPF*CH4_RF) ## Reference: CFR 1065.660
data_un["nint (Intake Air Flow)"] = data_un.get("Molar Flow Wet")/data_un.Mmix
data_un["xH2Ointdry"] = data_un.xH2Oint/(1-data_un.xH2Oint)
data_un["xH2Odildry"] = data_un.xH2Odil/(1-data_un.xH2Odil)
print("Fourth Level of Calculation finished!")

Fourth Level of Calculation finished!


In [8]:
##### Fifth Level of Calculation #####
data_un["xCO2int"] = xCO2intdry/(1+data_un.xH2Ointdry)
data_un["xO2int"] = ((0.20982-xCO2intdry)/(1+data_un.xH2Ointdry)) 
data_un["xCO2dil"] = xCO2dildry/(1+data_un.xH2Odildry)
print("Fifth Level of Calculation finished!")

Fifth Level of Calculation finished!


In [10]:
##### Circle Calculation #####
def f(variables):
    # Channels in Formulas
    #A = xdil/exh -- G
    #B = xH2Oexh -- H
    #C = xCcombdry -- I
    #D = xH2Oexhdry -- J
    #E = xdil/exhdry -- K
    #F = xint/exhdry -- L
    #G = xraw/exhdry -- M
    #H = xH2OCOmeas -- AC
    #I = xH2OTHCmeas -- AD
    #J = xH2ONOxmeas -- AE
    #K = xH2ONO2meas -- AF
    #L = xH2OCO2meas -- AH
    #M = xCOdry -- AI
    #N = xTHCdry -- AJ
    #O = xNOxdry -- AK
    #P = xNO2dry -- AL
    #Q = xCO2dry -- AN
    #R = xH2dry -- AO 
    
    # Unknown Variables
    (A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R) = variables
        
    # Known Variables
    S = data_un.xCO2dil[i]
    T = data_un.xCO2int[i]
    U = data_un.alpha[i]
    V = data_un.xH2Odil[i]
    W = data_un.xH2Oint[i]
    X = data_un.xO2int[i]
    Y = data_un.beta[i]
    Z = data_un.gamma[i]
    AA = data_un.delta[i]
    AB = data_un.xCOmeas[i]
    AC = data_un.xTHCmeas[i]
    AD = data_un.xNOxmeas[i]
    AE = data_un.xNO2meas[i]
    AF = data_un.xH2Ogas[i]
    AG = data_un.xCO2meas[i]
    
    # Equations to solve
    eq1 = 1-(G/(1+D))-A
    eq2 = D/(1+D)-B
    eq3 = Q+(M)+(N)-(S*E)-(T*F)-C
    eq4 = ((U/2)*(C-N))+V*E+W*F-R-D 
    eq5 = A/(1-B)-E
    eq6 = (1/(2*X))*(((U/2)-Y+2+(2*Z))*(C-N)-(M-O-(2*P)+R))-F
    eq7 = 0.5*(((U/2)+Y+AA)*(C-N)+((2*N)+M-P+R))+F-G
    eq8 = AB/(1-H)-M ## Reference: CFR 1065.659
    eq10 = AC/(1-I)-N ## Reference: CFR 1065.659
    eq11 = B-I
    eq12 = AD/(1-J)-O ## Reference: CFR 1065.659
    eq14 = AE/(1-K)-P ## Reference: CFR 1065.659
    eq16 = (M*(D-V*E))/(AF*(Q-S*E))-R ## Reference: CFR 1065.659
    eq17 = AG/(1-L)-Q ## Reference: CFR 1065.659
    
    if mode==0:
        eq9 = bs-H    
        eq13 = bs-J
        eq15 = bs-K
        eq18 = bs-L
    else:
        eq9 = B-H    
        eq13 = B-J
        eq15 = B-K
        eq18 = B-L
    
    return [eq1,eq2,eq3,eq4,eq5,eq6,eq7,eq8,eq9,eq10,eq11,eq12,eq13,eq14,eq15,eq16,eq17,eq18]

i = 0
mode = 0
result_list = np.zeros((len(data_un),18))

while i < len(data_un)-1:
    i +=1  
    g = 0.5 # First guess for iteration start

    solution = opt.fsolve(f,(g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g))
    
    if solution[2]<bs:
        mode = 1
        solution = opt.fsolve(f,(g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g))
        mode = 0
    result_list[:][i] = solution

iteration_df = pd.DataFrame(result_list,columns =['xdil/exh','xH2Oexh','xCcombdry','xH2Oexhdry','xdil/exhdry','xint/exhdry','xraw/exhdry','xH2OCOmeas','xH2OTHCmeas','xH2ONOxmeas','xH2ONO2meas','xH2OCO2meas','xCOdry','xTHCdry','xNOxdry','xNO2dry','xCO2dry','xH2dry'])
data_un = pd.concat([data_un,iteration_df],axis=1)

# Clear Variables
mode = None
result_list = None
g = None
solution = None
iteration_df = None
print("Circle Calculation finished!")
## 05/06/2015 - Anton lang, Lang@FEV.com

Circle Calculation finished!


In [12]:
##### Calculation of remaining values #####
data_un["xH2ONOmeas"] = data_un.xH2ONO2meas
data_un["xNOdry"] = data_un.xNOmeas/(1-data_un.xH2ONOmeas) ## Reference: CFR 1065.659
data_un["xCO2dry"] = data_un.xCO2meas/(1-data_un.xH2OCO2meas) ## Reference: CFR 1065.659
data_un["xH2dry"] = (data_un.xCOdry*(data_un.xH2Oexhdry-data_un.xH2Odil*data_un.get("xdil/exhdry")))/(data_un.xH2Ogas*(data_un.xCO2dry-data_un.xCO2dil*data_un.get("xdil/exhdry")))
data_un["xCOwet"] = data_un.xCOmeas*((1-data_un.xH2Oexh)/(1-data_un.xH2OCOmeas))
data_un['xNOxwet'] = data_un.xNOxmeas*((1-data_un.xH2Oexh)/(1-data_un.xH2ONOxmeas))
data_un["xNO2wet"] = data_un.xNO2meas*((1-data_un.xH2Oexh)/(1-data_un.xH2ONO2meas))
data_un["xNOwet"] = data_un.xNOmeas*((1-data_un.xH2Oexh)/(1-data_un.xH2ONOmeas))
data_un["xCO2wet"] = data_un.xCO2meas*((1-data_un.xH2Oexh)/(1-data_un.xH2OCO2meas))
data_un["xNOxcorrwet"] = data_un.xNOxwet*(18.84 *data_un.xH2O + 0.68094) ## Reference: CFR 1065.670

data_un["nexh"] = data_un.get("nint (Intake Air Flow)")/(1+((data_un.get("xint/exhdry")-data_un.get("xraw/exhdry"))/(1+data_un.xH2Oexhdry)))
data_un["Mass_THC"] = data_un.xTHCwet*data_un.nexh*data_un.get("Molar Mass of HC Emissions")
data_un["Mass_CO"] = data_un.xCOwet*data_un.nexh*data_un.get("Molar Mass of CO Emissions")
data_un["Mass_NOx"] = data_un.xNOxcorrwet*data_un.nexh*data_un.get("Molar Mass of NOx Emissions (as NO2)")
data_un["Mass_CO2"] = data_un.xCO2wet*data_un.nexh*data_un.get("Molar Mass of CO2 Emissions")
data_un["Mass_NMHC"] = data_un.xNMHCwet*data_un.nexh*data_un.get("Molar Mass of NMHC Emissions")

# Emissions in total
sum_THC = data_un.Mass_THC.sum(skipna = True)
sum_CO = data_un.Mass_CO.sum(skipna = True)
sum_NOx = data_un.Mass_NOx.sum(skipna=True)
sum_CO2 = data_un.Mass_CO2.sum(skipna=True)
sum_NMHC = data_un.Mass_NMHC.sum(skipna=True)
column_sum_un = [sum_CO2,sum_CO,sum_NOx,sum_THC,sum_NMHC]
print("Uncorrected Calculation finished!")

Uncorrected Calculation finished!


In [13]:
###############################################################
##################### Drift Correction ########################
###############################################################

In [57]:
##### Load Pre/Post - Files #####
import numpy as np
data_pre = pd.read_csv("Pre_zero_span.csv", low_memory=False)
data_pre = data_pre.convert_objects(convert_numeric=True)
data_post = pd.read_csv("Post_zero_span.csv", low_memory=False)
data_post = data_post.convert_objects(convert_numeric=True)
data_cor = pd.DataFrame()

species_pre = [col for col in list(data_pre) if col.startswith('E_')]

zero_span = pd.DataFrame(data=np.zeros([5,5]),index=['PreZero','PreSpan','PostZero','PostSpan','Chosen'],columns=['E_CO2D2',CO,'E_NOXD2','E_THCW2','E_CH4W2'])
print('Data Load finished!')

Data Load finished!


In [61]:
zero_span

,E_CO2D2,E_COHD2,E_NOXD2,E_THCW2,E_CH4W2
PreZero,0,0,0.500,0.315,0.000
PreSpan,190040,58800,4064.960,9900.020,2654.130
PostZero,40,0,0.989,0.162,0.162
PostSpan,189960,58840,4052.060,9891.210,2651.450
Chosen,191000,58700,4005.000,9900.000,2510.000


In [58]:
### Pre-File
zero_data = data_pre.loc[59, species_pre[0]:species_pre[-1]]
span_data = data_pre.loc[120, species_pre[0]:species_pre[-1]]

## Zero -  Data ##
zero_span.E_CO2D2['PreZero'] = abs(zero_data.E_CO2D2+0) * 10000
zero_span[CO]['PreZero'] = abs(zero_data.get(CO)+0) * 10000
zero_span.E_NOXD2['PreZero'] = abs(zero_data.E_NOXD2+0)
zero_span.E_THCW2['PreZero'] = abs(zero_data.E_THCW2+0)

## Span - Data ##
zero_span.E_CO2D2['PreSpan'] = abs(data_pre.E_CO2D2.iloc[-1]+0) * 10000
zero_span[CO]['PreSpan'] = abs(span_data.get(CO)+0) * 10000
zero_span.E_NOXD2['PreSpan'] = abs(span_data.E_NOXD2+0)
zero_span.E_THCW2['PreSpan'] = abs(span_data.E_THCW2+0)
zero_span.E_CH4W2['PreSpan'] = abs(data_pre.E_THCW2.iloc[-1]+0)
print('Pre-Zero/Span is finished!')

Pre-Zero/Span is finished!


In [60]:
### Post-File
zero_data = data_post.loc[59, species_pre[0]:species_pre[-1]]
span_data = data_post.loc[120, species_pre[0]:species_pre[-1]]

## Zero -  Data ##
zero_span.E_CO2D2['PostZero'] = abs(zero_data.E_CO2D2+0) * 10000
zero_span[CO]['PostZero'] = abs(zero_data.get(CO)+0) * 10000
zero_span.E_NOXD2['PostZero'] = abs(zero_data.E_NOXD2+0)
zero_span.E_THCW2['PostZero'] = abs(zero_data.E_THCW2+0)
zero_span.E_CH4W2['PostZero'] = abs(zero_data.E_THCW2+0)

## Span - Data ##
zero_span.E_CO2D2['PostSpan'] = abs(data_post.E_CO2D2.iloc[-1]+0) * 10000
zero_span[CO]['PostSpan'] = abs(span_data.get(CO)+0) * 10000
zero_span.E_NOXD2['PostSpan'] = abs(span_data.E_NOXD2+0)
zero_span.E_THCW2['PostSpan'] = abs(span_data.E_THCW2+0)
zero_span.E_CH4W2['PostSpan'] = abs(data_post.E_THCW2.iloc[-1]+0)

## Chosen Values - From Database (Tanks)
zero_span.E_CO2D2['Chosen'] = 191000
zero_span[CO]['Chosen'] = 58700
zero_span.E_NOXD2['Chosen'] = 4005
zero_span.E_THCW2['Chosen'] = 9900
zero_span.E_CH4W2['Chosen'] = 2510
print('Post-Zero/Span is finished!')

Post-Zero/Span is finished!


In [ ]:
## Correct Raw-Emissions ##
data.E_CO2D2 = zero_span.E_CO2D2['Chosen']*((2*data_un.E_CO2D2)-(zero_span.E_CO2D2['PreZero']+zero_span.E_CO2D2['PostZero']))/((zero_span.E_CO2D2['PreSpan']+zero_span.E_CO2D2['PostSpan'])-(zero_span.E_CO2D2['PreZero']+zero_span.E_CO2D2['PostZero']))
data.E_COHD2 = zero_span.E_COHD2['Chosen']*((2*data_un.E_COHD2)-(zero_span.E_COHD2['PreZero']+zero_span.E_COHD2['PostZero']))/((zero_span.E_COHD2['PreSpan']+zero_span.E_COHD2['PostSpan'])-(zero_span.E_COHD2['PreZero']+zero_span.E_COHD2['PostZero']))
data.E_NOXD2 = zero_span.E_NOXD2['Chosen']*((2*data_un.E_NOXD2)-(zero_span.E_NOXD2['PreZero']+zero_span.E_NOXD2['PostZero']))/((zero_span.E_NOXD2['PreSpan']+zero_span.E_NOXD2['PostSpan'])-(zero_span.E_NOXD2['PreZero']+zero_span.E_NOXD2['PostZero']))
data.E_THCW2 = zero_span.E_THCW2['Chosen']*((2*data_un.E_THCW2)-(zero_span.E_THCW2['PreZero']+zero_span.E_THCW2['PostZero']))/((zero_span.E_THCW2['PreSpan']+zero_span.E_THCW2['PostSpan'])-(zero_span.E_THCW2['PreZero']+zero_span.E_THCW2['PostZero']))
data.E_CH4W2 = zero_span.E_CH4W2['Chosen']*((2*data.E_CH4W2)-(zero_span.E_CH4W2['PreZero']+zero_span.E_CH4W2['PostZero']))/((zero_span.E_CH4W2['PreSpan']+zero_span.E_CH4W2['PostSpan'])-(zero_span.E_CH4W2['PreZero']+zero_span.E_CH4W2['PostZero']))
print('Raw Emissions drift-corrected!')